# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityCsv = "city_weather.csv"
cityWeatherDF = pd.read_csv("city_weather.csv")
cityWeatherDF.head()

,Unnamed: 0,CityID,CityName,Country,Lat,Lng,Temperature,MaxTemp,Humidity,Cloudiness,WindSpeed
0,0,1282256,Hithadhoo,MV,-0.60,73.08,83.12,83.12,73,76,9.80
1,1,4031637,Lavrentiya,RU,65.58,-171.00,9.12,9.12,95,100,9.51
2,2,2094342,Kavieng,PG,-2.57,150.80,84.22,84.22,73,75,21.68
3,3,3369157,Cape Town,ZA,-33.93,18.42,66.13,66.99,56,42,8.01
4,4,933995,Souillac,MU,-20.52,57.52,80.01,80.01,86,80,1.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

locations = cityWeatherDF[["Lat", "Lng"]]
humidity = cityWeatherDF["Humidity"].astype(float)

fig = gmaps.figure()

heatLayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius=3)
fig.add_layer(heatLayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
maxTemperatureFilterDF = cityWeatherDF[(cityWeatherDF['MaxTemp'] < 90.0) & (cityWeatherDF['MaxTemp'] > 70.0)]
maxTempHumidityFilterDF = maxTemperatureFilterDF[(maxTemperatureFilterDF['Humidity']<60) & (maxTemperatureFilterDF['Humidity']>20)]
perfectVacationDF = maxTempHumidityFilterDF[(maxTempHumidityFilterDF['Cloudiness'] < 50)].dropna()
perfectVacationDF

,Unnamed: 0,CityID,CityName,Country,Lat,Lng,Temperature,MaxTemp,Humidity,Cloudiness,WindSpeed
20,20,5848280,Kapaa,US,22.08,-159.32,76.95,78.80,57,40,16.11
30,30,1651810,Airai,TL,-8.93,125.41,75.94,75.94,58,10,2.71
50,50,2075265,Busselton,AU,-33.65,115.33,75.45,79.00,31,0,7.34
220,220,3689026,Becerril,CO,9.70,-73.28,85.33,85.33,56,45,4.81
309,309,3430863,Mar del Plata,AR,-38.00,-57.56,79.27,80.01,47,0,9.17
360,360,2140466,Moindou,NC,-21.69,165.68,87.80,87.80,45,20,11.41
416,416,2338660,Yaan,NG,7.38,8.57,77.23,77.23,29,2,5.53
420,420,6355222,Yulara,AU,-25.24,130.99,84.20,84.20,23,35,18.34
469,469,3703938,Nata,PA,8.33,-80.52,78.01,78.01,41,5,3.00
484,484,3620674,Boaco,NI,12.47,-85.66,86.00,86.00,58,40,11.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotelDF = pd.DataFrame(columns=['Hotel Name', 'CityID'])

for index,row in perfectVacationDF.iterrows():
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_search = "Hotels"
    target_radius = 5000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response = response.json()
    if response['status'] =='OK':
        hotelDF = hotelDF.append({'Hotel Name': response["results"][0]["name"],
                        'CityID': row['CityID']}, ignore_index=True)
    else:
        print('No hotels for requested City')

hotelWeatherMergeDF = pd.merge(hotelDF,perfectVacationDF, on='CityID')
hotelWeatherMergeDF = hotelWeatherMergeDF[['CityID', 'CityName', 'Country', 'Lat', 'Lng', 'Hotel Name', 'Temperature', 'MaxTemp', 'Humidity', 'Cloudiness', 'WindSpeed']]
hotelWeatherMergeDF.set_index('CityID', inplace=True)
hotelWeatherMergeDF

No hotels for requested City
No hotels for requested City
No hotels for requested City
No hotels for requested City


,CityName,Country,Lat,Lng,Hotel Name,Temperature,MaxTemp,Humidity,Cloudiness,WindSpeed
CityID,,,,,,,,,,
5848280,Kapaa,US,22.08,-159.32,Sheraton Kauai Coconut Beach Resort,76.95,78.80,57,40,16.11
2075265,Busselton,AU,-33.65,115.33,Abbey Beach Resort,75.45,79.00,31,0,7.34
3689026,Becerril,CO,9.70,-73.28,Hotel Y Variedades Sharay.,85.33,85.33,56,45,4.81
3430863,Mar del Plata,AR,-38.00,-57.56,Sheraton Mar del Plata Hotel,79.27,80.01,47,0,9.17
6355222,Yulara,AU,-25.24,130.99,Sails in the Desert,84.20,84.20,23,35,18.34
3620674,Boaco,NI,12.47,-85.66,Hotel Tijerinos,86.00,86.00,58,40,11.41
3628473,San Cristobal,VE,7.77,-72.22,Lidotel Hotel Boutique San Cristobal,82.40,82.40,54,40,4.70
2132606,Samarai,PG,-10.62,150.67,Nuli Sapi,88.39,88.39,57,49,5.21


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{CityName}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelWeatherMergeDF.iterrows()]
locations = hotelWeatherMergeDF[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display Map
fig.add_layer(markerLayer)
fig

Figure(layout=FigureLayout(height='420px'))